In [1]:
import pandas as pd
import numpy as np


In [7]:
proff_df = pd.read_excel('400-professions.xlsx')
vacancy_df = pd.read_csv('v.csv')

In [8]:
import re
from bs4 import BeautifulSoup as bs
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

def preprocess(sentence):
    soup = bs(sentence, features="html.parser")
    sentence = soup.get_text()
    soup = bs(sentence, features="html.parser")
    sentence = soup.get_text()

    sentence = str(sentence)
    sentence = sentence.lower()
    sentence = sentence.replace('{html}',"")
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url = re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)
    filtered_words = [w for w in tokens if not w in stopwords.words('russian')]

    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mauka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
# !python -m spacy download ru_core_news_sm

In [20]:
nlp = spacy.load('ru_core_news_sm')

In [21]:
vacancy_df['cleaned_vacancy_description'] = vacancy_df['clean_description'].apply(preprocess)
proff_df['Наименование КК_clean']= proff_df['Наименование КК'].apply(preprocess)

C:\Users\Mauka\AppData\Local\Temp\ipykernel_18140\2697063262.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(sentence, features="html.parser")
C:\Users\Mauka\AppData\Local\Temp\ipykernel_18140\2697063262.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(sentence, features="html.parser")


In [22]:
cleaned_professions = proff_df['Наименование КК_clean'].tolist()
cleaned_vacancy_descriptions = vacancy_df['cleaned_vacancy_description'].tolist()

In [23]:
texts = cleaned_professions + cleaned_vacancy_descriptions

In [24]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)

In [25]:
similarity_matrix = cosine_similarity(tfidf_matrix)
professions_similarity = similarity_matrix[:len(cleaned_professions), len(cleaned_professions):]
most_similar_professions_indices = professions_similarity.argmax(axis=0)
most_similar_professions = [cleaned_professions[idx] for idx in most_similar_professions_indices]
vacancy_df['most_similar_profession'] = most_similar_professions

In [26]:
vacancy_df[['name', 'most_similar_profession']][350:400]

,name,most_similar_profession
350,Торговый представитель по аптекам и магазинам,менеджер продажам
351,Android Разработчик (Удаленно),разработчик мобильных приложений
352,Уборщик,ветеринарный врач
353,Контролер ОТК,менеджер системе менеджмента качества
354,Контролер службы внутренней безопасности,ветеринарный врач
355,Оператор вахтер,ветеринарный врач
356,Junior Category Manager,ветеринарный врач
357,Помощник бухгалтера по расчету заработной платы,бухгалтер налогам
358,Начальник дорожно-строительного участка,техник дорожного строительства
359,Специалист по снабжению (Снабженец),техник аэродромной службы


In [28]:
# !pip install sentence-transformers

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2', device=device)

profession_embeddings = model.encode(cleaned_professions, convert_to_tensor=True, show_progress_bar=True, device=device)
vacancy_embeddings = model.encode(cleaned_vacancy_descriptions, convert_to_tensor=True, show_progress_bar=True, device=device)

similarity_matrix_bert = util.pytorch_cos_sim(profession_embeddings, vacancy_embeddings)

most_similar_professions_indices_bert = similarity_matrix_bert.argmax(dim=0)
most_similar_professions_bert = [cleaned_professions[idx] for idx in most_similar_professions_indices_bert]

vacancy_df['most_similar_profession_bert'] = most_similar_professions_bert


In [29]:
match_vacancy_df = pd.read_csv('vacancy_df_after_matching.csv')

In [31]:
proff_df['Наименование КК'] = proff_df['Наименование КК'].apply(preprocess)

C:\Users\Mauka\AppData\Local\Temp\ipykernel_18140\2697063262.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(sentence, features="html.parser")
C:\Users\Mauka\AppData\Local\Temp\ipykernel_18140\2697063262.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(sentence, features="html.parser")


In [32]:
it_ikt_professions = proff_df[proff_df['сфера'].str.contains('ИТ и ИКТ', regex=True, na=False)]['Наименование КК'].tolist()

In [34]:
def update_it_tag(row, it_ikt_professions):
    if row['most_similar_profession'] in it_ikt_professions:
        return row['most_similar_profession']
    return row['most_similar_profession_bert']

match_vacancy_df['most_similar_profession_bert'] = match_vacancy_df.apply(update_it_tag, axis=1, it_ikt_professions=it_ikt_professions)

In [35]:
vacancy_df = pd.read_csv('vacancy_df_updated.csv')

In [41]:
vacancy_df[['name','most_similar_profession_bert']][250:600]

,name,most_similar_profession_bert
250,Инженер ПТО,инспектор техническому обслуживанию зданий
251,Грузчик-комплектовщик,менеджер системе менеджмента качества
252,Специалист АХЧ,инспектор техническому обслуживанию зданий
253,Торговый представитель (строительные материалы),менеджер работе клиентами
254,Торговый представитель (строительные материалы),менеджер работе клиентами
...,...,...
595,"Менеджер по работе с клиентами, оператор на CA...",менеджер работе клиентами
596,Региональный менеджер по работе с клиентами,консультанты профориентации
597,Региональный представитель (TM VARTON),инженер организации эксплуатации ремонту
598,Оператор фронтального погрузчика SDLG,инспектор дорожный


In [36]:
class_distribution = vacancy_df['most_similar_profession_bert'].value_counts()
print(class_distribution)

менеджер работе клиентами                     1334
прочие сотрудники сфере бизнеса               1095
инспектор техническому обслуживанию зданий     527
инженер организации нормированию труда         504
инспектор труду сфере промышленности           464
                                              ... 
техники биологи                                  1
научный сотрудник промышленности                 1
лаборант рад защита                              1
экономист труду                                  1
механик судовой                                  1
Name: most_similar_profession_bert, Length: 239, dtype: int64


# **option with rubert**

In [1]:
!pip install transformers
!pip install torch

  Using cached transformers-4.42.4-py3-none-any.whl (9.3 MB)
  Using cached huggingface_hub-0.24.0-py3-none-any.whl (419 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl (177 kB)


You should consider upgrading via the 'c:\users\mauka\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\mauka\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512, padding='max_length')
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return cls_embedding

In [7]:
def extract_key_sentences(text, max_tokens=512):

    sentences = text.split('. ')

    if len(tokenizer(text)['input_ids']) <= max_tokens:
        return text

    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentences)
    scores = X.sum(axis=1).A1

    ranked_sentences = [sentences[i] for i in np.argsort(scores)[::-1]]

    selected_sentences = []
    total_tokens = 0
    for sentence in ranked_sentences:
        sentence_tokens = len(tokenizer(sentence)['input_ids'])
        if total_tokens + sentence_tokens <= max_tokens:
            selected_sentences.append(sentence)
            total_tokens += sentence_tokens
        else:
            break

    return '. '.join(selected_sentences)

In [11]:
vacancy_df = pd.read_csv('v.csv')
proff_df = pd.read_excel('400-professions.xlsx')

In [15]:
proff_df['Наименование КК_clean'] = proff_df['Наименование КК'].apply(preprocess)

C:\Users\Mauka\AppData\Local\Temp\ipykernel_19884\2697063262.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(sentence, features="html.parser")
C:\Users\Mauka\AppData\Local\Temp\ipykernel_19884\2697063262.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(sentence, features="html.parser")


In [26]:
import numpy as np

In [29]:
from tqdm import tqdm

In [34]:
vacancy_df['key_sentences'] = vacancy_df['clean_description'].apply(lambda x: extract_key_sentences(x))
vacancy_df['num_tokens'] = vacancy_df['key_sentences'].apply(lambda x: len(tokenizer(x)['input_ids']))

In [37]:
vacancy_df['num_tokens'] = list(tqdm(vacancy_df['key_sentences'].apply(lambda x: len(tokenizer(x)['input_ids']))))
proff_df['embedding'] = list(tqdm(proff _df['Наименование КК_clean'].apply(get_bert_embedding)))
vacancy_df['embedding'] = list(tqdm(vacancy_df['key_sentences'].apply(get_bert_embedding)))

100%|██████████| 403/403 [00:00<00:00, 403029.21it/s]


: 

In [125]:
def find_best_match(vacancy_embedding, profession_embeddings, profession_names):
    similarity_scores = cosine_similarity([vacancy_embedding], profession_embeddings)[0]
    best_match_index = similarity_scores.argmax()
    return profession_names[best_match_index]

vacancy_df['best_profession_match'] = vacancy_df['embedding'].apply(
    lambda x: find_best_match(x, proff_df['embedding'].tolist(), proff_df['Наименование КК_clean'].tolist())
)


In [187]:
# vacancy_df[['name', 'best_profession_match']][:400]

In [170]:
# import pandas as pd
# from transformers import BertTokenizer

# tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

# def count_tokens(text):
#     inputs = tokenizer(text, truncation=True)
#     return len(inputs['input_ids'])

# text = 'техник пусконаладочным работам классифицируется специалист нанят уволен работы приказу генерального директора должность техник вводу эксплуатацию назначается лицо имеющее высшее профессиональное техническое образование опыт работы качестве специалиста вводу эксплуатацию менее лет технический специалист вводу эксплуатацию подотчетен непосредственно инженеру вводу эксплуатацию ведущему инженеру электрику силу специфики своей работы техник вводу эксплуатацию должен знать положения директивы приказы инструкции нормативные материалы рк организации ввода эксплуатацию основные технологические процессы производства предприятии технические условия конструктивные особенности назначение режимы работы оборудования предприятия правила технической эксплуатации методы монтажа наладки настройки электрооборудования контрольные инструменты приборы оборудование используемые осмотре вводе эксплуатацию испытаниях обслуживаемого оборудования опыт работы документацией электрической конфигурации однолинейные чертежи электрические схемы технические руководства испытание ввод эксплуатацию распределительного устройства электрораспределительного оборудования mv lv защитных компонентов пусконаладочные работы испытания подача питания панели шкафы ибп цепи зарядных устройств тестирование подача питания бд освещения малой мощности силовые трансформаторы измерительные приборы также базы данных телекоммуникаций испытания подача электропитания полевых потребителей электроэнергии двигатели клапаны нагреватели т д организация монтажных пусконаладочных ремонтных работ техническое обслуживание оборудования поддержка устранения неполадок электрических проблем связанных электрическим оборудованием кабелями компонентами проверка результатов пусконаладочных испытаний обеспечения безопасной эксплуатации электрической распределительной системы лучшие отечественные зарубежные практики безопасности области ввода эксплуатацию основы трудового законодательства правила нормы охраны труда умение взаимодействовать заказчиком многопрофильными специалистами компании подрядчиками культурная осведомленность способность работать людьми разных национальностей знакомство системой разрешение работу сводки инструментов специфические требования площадкиорганизовать выполнить пусконаладочные работы видов оборудования соответствии методическим нормативным иным инструктивным материалом организации пусконаладочных работ обеспечить своевременный ввод эксплуатацию подготовить измерительные приборы аппаратуру работе выполнить метрологический контроль начать работы устранению дефектов оборудования выявленных вводе эксплуатацию храните техническую документацию монтаже вводе эксплуатацию тестировании оборудования оказание помощи подготовке документации передаче досье наценки т д контролировать сотрудников участвующих вводе эксплуатацию обеспечивать соблюдение требований охраны труда соответствии видом выполняемых работ требования высшее либо средне техническое образование области электрики менее х лет опыта работы пуско наладке оборудовании нефтегазовых месторождениях специфике своей деятельности техник электрик пусконаладке должен знать постановления распоряжения приказы методические нормативные материалы вопросам организации пусконаладочных работ электрооборудования основные технологические процессы технические характеристики конструктивные особенности назначение режимы работы оборудования правила технической эксплуатации методы монтажа регулировки наладки электрического энергетического оборудования контрольные средства приборы устройства применяемые проверке наладке испытаниях обслуживаемого электрического энергетического оборудования порядок методы планирования монтажных наладочных испытательных работ организацию монтажных пусконаладочных ремонтных работ проведения технического обслуживания электрического энергетического оборудования передовой отечественный зарубежный опыт области проведения пусконаладочных работ порядок составления смет проведение работ заявок оборудование материалы запасные части измерительные инструменты приборы порядок разработки оформления технической документации основы организации производства труда управления основы трудового законодательства правила нормы охраны труда организует выполняет работу попусконаладке видов оборудования соответствии методическими нормативными другими руководящими материалами организации пусконаладочных работ обеспечивает своевременный ввод эксплуатацию составляет программы календарные графики проведения пусконаладочных работ испытаний согласовывает заказчиками осуществляет подготовку работе средств измерений аппаратуры выполняет метрологический контроль организует работу персонала обеспечивает рациональное расходование сырья материалов необходимых ввода оборудования систем эксплуатацию обеспечения хода технологического процесса период пусконаладочных работ организует обеспечивает качество ведения работ вносит необходимые коррективы способы методы наладки целью достижения необходимых параметров характеристик работы оборудования систем производит регулировку анализирует данные измерений параметров работы выполняет необходимые расчеты дает заключения пригодности эксплуатации отдельных деталей узлов механизмов систем выявляет причины неисправности инициирует работы устранению дефектов оборудования выявленных выполнении пусконаладочных работ условия вахта проживание питание проезд предоставляет компания'
# num_tokens = count_tokens(text)
# print(f"Количество токенов: {num_tokens}")

In [169]:
# long_descriptions = vacancy_df[vacancy_df['word_count'] > 400][['clean_description', 'word_count']]
# print(f"Количество описаний со словами больше 400: {len(long_descriptions)}")
# print(long_descriptions)


In [150]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

import nltk
nltk.download('punkt')
def summarize_text(text, sentence_count=10):
    parser = PlaintextParser.from_string(text, Tokenizer("russian"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentence_count)
    return ' '.join([str(sentence) for sentence in summary])

vacancy_df['summarized_description'] = vacancy_df['cleaned_vacancy_description'].apply(summarize_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [174]:
vacancy_df['num_tokens'] = vacancy_df['summarized_description'].apply(count_tokens)

In [168]:
# long_token = vacancy_df[vacancy_df['num_tokens'] > 512][['summarized_description', 'num_tokens']]
# long_token